# Data Collection
This notebook is dedicated to the collection and insertion of data into a MySQL database hosted on Google Cloud. The collection code will ultimately be transferred to a python module file that will be saved in the same subfolder as this document.

In [2]:
import config
import mysql.connector 
import googlemaps
from datetime import datetime
from mysql.connector import errorcode

%load_ext autoreload
%autoreload 2

## Google Maps Directions API

API documentation available [here](https://developers.google.com/maps/documentation/directions/start?hl=en_US).

Python client for maps API available [here](https://github.com/googlemaps/google-maps-services-python)

In [3]:
gmaps = googlemaps.Client(key=config.api_key)

# Geocoding my address
geocode_my_result = gmaps.geocode(config.my_address)

# Store just latitude and longitude
my_geometry = geocode_my_result[0]['geometry']['location']

# Geocoding my girlfriend's address
geocode_gf_result = gmaps.geocode(config.gf_address)

# Store just latitude and longitude
gf_geometry = geocode_gf_result[0]['geometry']['location']

In [24]:
class GDirections:
    """
    A class to keep track of and display to the user only necessary parts of the Google maps searches.
    """

    
    def __init__(self, start_geometry, end_geometry, mode):
        
        """
        Parameters
        -----------
    
        start_geometry: latitude and longitude of start location
        end_geometry: latitude and longitude of end location
        mode: One of "driving", "walking", "bicycling" or "transit"
        """
        
        self._trip_start = datetime.now()
        self._directions_json = self._retrieve_google_directions_json(start_geometry, end_geometry, mode)
        self._trip_duration = self._calculate_trip_duration()
        
        
    def _retrieve_google_directions_json(self,start_geometry, end_geometry, mode):
        
        """
        Call Google Maps API to retrieve a json with directions information
        """
        
        # Call API
        directions = gmaps.directions(
            origin = start_geometry,
            destination = end_geometry,
            mode = mode,
            departure_time = self._trip_start
        )
        return directions
    
    def _calculate_trip_duration(self):
        
        """
        Calculate and return trip duration in minutes.
        
        Returns
        -------
        
        Trip duration as a string in minutes.
        
        e.g. '50 minutes'
        """
        
        # Store arrival time and convert to datetime
        arrival_time = self._directions_json[0]['legs'][0]['arrival_time']['text']
        arrival_datetime = datetime.strptime(arrival_time, '%H:%M%p')
    
        # Store arrival time and convert to datetime
        departure_time = self._directions_json[0]['legs'][0]['departure_time']['text']
        departure_datetime = datetime.strptime(departure_time, '%H:%M%p')
        
        # When we go from 12 am/pm -> 1am/pm, we don't want to say the difference is 11 hours
        # Thus, account for when departure hour is greater than arrival hour
        if departure_datetime.hour > arrival_datetime.hour:
            departure_datetime = departure_datetime.replace(hour=0)
        
        # Calculate difference in seconds, convert to minutes
        trip_duration = int( (arrival_datetime - departure_datetime).seconds/60 )
        
        return trip_duration
    
    def get_trip_duration(self):
        """
        Getter method for trip_duration in minutes as int
        """
        
        return self._trip_duration
    
    def get_trip_instructions(self):
        """
        RETURNS
        -------
        A dictionary with which trains/buses to take in this trip and how long each one will take.

        PARAMETERS
        ----------

        directions: [JSON] raw directions contained within self._directions_json
        """
        # Empty container for results
        directions_list = []
    
        # Isolate just the specific trip directions
        directions = self._directions_json[0]['legs'][0]['steps']
        
        # keep track of which step we're in
        step = 1
        
        for i, direction in enumerate(directions):
            
            # Ignore any walking instructions
            if direction.get('transit_details'):
                
                # Store name of bus or train line
                travel_mode = direction['transit_details']['line']['short_name']
                
                # Store time on that bus or train line in minutes as int
                direction_len = int(direction['duration']['text'].split()[0])
                
                # Append dict to the results container
                directions_list.append({
                    'step':step,
                    'travel_mode' : travel_mode,
                    'length' : direction_len
                })
                
                # Next step will be 1 higher
                step += 1

        return directions_list
    
    def get_trip_start(self):
        return self._trip_start.strftime("%Y/%m/%d %H:%M")

In [ ]:
# Trips to gf's home
transit_to_gf = GDirections(my_geometry, gf_geometry, 'transit')

# Trips to my home
transit_to_me = GDirections(gf_geometry, my_geometry, 'transit')

In [37]:
# Store these together in a list
trip_objects = [transit_to_gf, transit_to_me]

In [40]:
trip_objects

[<__main__.GDirections at 0x11f042a10>, <__main__.GDirections at 0x11eff6b90>]

In [12]:
trip_objects[0].get_trip_start()

'2020/02/17 15:39'

In [13]:
trip_objects[0].get_trip_duration()

66

In [26]:
trip_objects[0].get_trip_instructions()

[{'step': 1, 'travel_mode': 'Q59', 'length': 32},
 {'step': 2, 'travel_mode': 'R', 'length': 5}]

In [27]:
# #Convert date string back to datetime
# #This will be used to convert the string back to datetime object after we pull back down from db

# datetime.strptime(db_time, "%Y/%m/%d %H:%M")

# Set up MySQL database
Automate data collection to add trip information every 5 mins or so.

In [29]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)
cursor = cnx.cursor()

In [11]:
cursor.close()
cnx.close()

In [31]:
## Create new DB 
db_name = 'google_maps'

def create_database(cursor, db_name):
    """
    Creates a MySQL database.
    
    PARAMETERS
    ----------
    
    cursor: cursor for a given mysql connection object
    db_name: [str] name of the new database
    """
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(db_name))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))


try:
    # Check to see if db exists
    cursor.execute("USE {}".format(db_name))
    
except mysql.connector.Error as err:
    
    print("Database {} does not exists.".format(db_name))
    
    # If db doesn't exist, create it
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        
        create_database(cursor, db_name)
        
        print("Database {} created successfully.".format(db_name))
        
        cnx.database = db_name
    
    # If another error, print it
    else:
        print(err)

In [77]:
# Create a table for the data

create_query = """
CREATE TABLE trips (
    departure_time varchar(20) NOT NULL,
    trip_direction int(5) NOT NULL,
    trip_duration int(10) NOT NULL,
    PRIMARY KEY (departure_time, trip_direction)
    );

"""
cursor.execute(create_query)

In [78]:
# Create a table for the data

create_query = """
CREATE TABLE instructions (
    departure_time varchar(20) NOT NULL,
    trip_direction int(5) NOT NULL,
    trip_step int(5) NOT NULL,
    transit_line varchar(5) NOT NULL,
    step_duration int(5) NOT NULL
    );

"""
cursor.execute(create_query)

In [79]:
# write a function to insert data into the reviews table
def insert_trip_data(trip_objects, cursor, cnx):
    """
    Inserts data into google_maps.trips table.
    
    PARAMETERS
    ----------
    
    trip_object: List of 2 GDirections trip objects containing information 
                 to be inserted into the MySQL table. First object contains
                 info from my house to gf and second object is the other trip.
    """
    # Create list of tuples for the objects in trip_objects
    trip_tuples = [
        (
        trip_object.get_trip_start(),         # departure_time
        i,                                    # trip_direction (either 0 or 1)
        trip_object.get_trip_duration(),      # trip_duration
        )
        for i, trip_object in enumerate(trip_objects)
    ]
       
    insert_statement = """
                       INSERT INTO 
                         trips (departure_time, trip_direction, trip_duration) 
                       VALUES 
                         (%s, %s, %s)
                       ;"""
    cursor.executemany(insert_statement,trip_tuples)
    cnx.commit()
    
def insert_instructions_data(trip_objects, cursor, cnx):
    """
    Inserts data into google_maps.instructions table.
    
    PARAMETERS
    ----------
    
    trip_object: List of 2 GDirections trip objects containing information 
                 to be inserted into the MySQL table. First object contains
                 info from my house to gf and second object is the other trip.
    """
    final_tuples = []
    
    # Start by iterating through trip_objects
    for i, trip_object in enumerate(trip_objects):
        
        # Create list of tuples for the objects in trip_objects
        instructions_tuples = [
            (
                trip_object.get_trip_start(),    # departure_time
                i,                               # trip_direction (either 0 or 1)
                trip_step['step'],               # trip_setp
                trip_step['travel_mode'],        # transit_line
                trip_step['length']             # step_duration
            )
            for trip_step in trip_object.get_trip_instructions()
        ]
        
        # Append to final_tuples list
        final_tuples.extend(instructions_tuples)
       
    insert_statement = """
                       INSERT INTO 
                         instructions (departure_time, trip_direction, trip_step, transit_line, step_duration) 
                       VALUES 
                         (%s, %s, %s, %s, %s)
                       ;"""
    cursor.executemany(insert_statement,final_tuples)
    cnx.commit()


In [80]:
insert_trip_data(trip_objects, cursor, cnx)

insert_instructions_data(trip_objects, cursor, cnx)